# Property_graph using Multiple GPU
#### Author : Don Acosta

In this notebook, we will show how to create and use a Property Graph using multiple GPUs 

This notebook was developed and tested using RAPIDS 23.02 and CUDA 11.5. Please be aware that your system may be different, and you may need to modify the code or install packages to run the below examples. If you think you have found a bug or an error, please file an issue in [cuGraph](https://github.com/rapidsai/cugraph/issues)


CuGraph's multi-GPU features leverage Dask. RAPIDS has other projects based on Dask such as dask-cudf and dask-cuda. These products will also be used in this example. Check out [RAPIDS.ai](https://rapids.ai/) to learn more about these technologies.

## Multi GPU Louvain with cuGraph
### Basic setup

In [1]:
# Import needed libraries. We recommend using the [cugraph_dev](https://github.com/rapidsai/cugraph/tree/branch-21.12/conda/environments) env through conda
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from cugraph.dask.comms import comms as Comms
import cugraph.dask as dask_cugraph
import cugraph
import dask_cudf
import time

In [2]:
import urllib.request
import os

data_dir = '../data/'
if not os.path.exists(data_dir):
    print('creating data directory')
    os.system('mkdir ../data')

# download the Hollywood dataset
base_url = 'https://rapidsai-data.s3.us-east-2.amazonaws.com/cugraph/benchmark/'
fn = 'hollywood.csv'
comp = '.gz'

if not os.path.isfile(data_dir+fn):
    if not os.path.isfile(data_dir+fn+comp):
        print(f'Downloading {base_url+fn+comp} to {data_dir+fn+comp}')
        urllib.request.urlretrieve(base_url+fn+comp, data_dir+fn+comp)
    print(f'Decompressing {data_dir+fn+comp}...')
    os.system('gunzip '+data_dir+fn+comp)
    print(f'{data_dir+fn+comp} decompressed!')
else:
    print(f'Your data file, {data_dir+fn}, already exists')

# File path, assuming Notebook directory
input_data_path = data_dir+fn

Your data file, ../data/hollywood.csv, already exists


### Initialize multi-GPU environment
Before we get started, we need to setup a Dask local cluster of workers to execute our work and a client to coordinate and schedule work for that cluster. As we see below, we can initiate a cluster and client using only 3 lines of code.

In [3]:
cluster = LocalCUDACluster()
client = Client(cluster)
Comms.initialize(p2p=True)

2023-01-09 19:28:46,584 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-01-09 19:28:46,584 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-01-09 19:28:46,585 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-01-09 19:28:46,585 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


### Read the data from disk
cuGraph depends on cudf for data loading and the initial DataFrame creation. The CSV data file contains an edge list, which represents the connection of a vertex to another. The source to destination pairs is what is known as Coordinate Format (COO). In this test case, the data is just two columns. 

In [4]:
# Start ETL timer
t_start = time.time()

# Helper function to set the reader chunk size to automatically get one partition per GPU  
chunksize = dask_cugraph.get_chunksize(input_data_path)

# Multi-GPU CSV reader
e_list = dask_cudf.read_csv(input_data_path, chunksize = chunksize, delimiter=' ', names=['src', 'dst'], dtype=['int32', 'int32'])


### Create a graph

In [5]:
from cugraph.experimental import MGPropertyGraph
pG = MGPropertyGraph()
pG.add_edge_data(e_list,vertex_col_names=["src", "dst"])
pG.get_num_edges()

113891327

In [13]:
graph = cugraph.Graph(directed=False) 
(louvain_df, modularity) = dask_cugraph.louvain(pG.extract_subgraph(create_using=graph))
pG.add_vertex_data(louvain_df, vertex_col_name="vertex")
G0 = pG.extract_subgraph(selection=pG.select_vertices("partition == 0")) 
pageranks0 = cugraph.pagerank(G0)
print(pageranks0.sort_values (by="pagerank", ascending=False).head(3)) 


NotImplementedError: 

In [ ]:
Comms.destroy()
client.close()
cluster.close()

___
Copyright (c) 2023, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___